In [4]:
!pip install -q faster-whisper flask pyngrok pysocks soundfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 14.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 16.2 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 52.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.6 MB/s eta 0:00:00


In [ ]:
from faster_whisper import WhisperModel

print("Loading ASR model...")

model = WhisperModel("base", device="cpu", compute_type="int8")
# model = WhisperModel("large-v3", device="cuda", compute_type="float16")

print("Model loaded successfully!")

In [ ]:
from flask import Flask, request, jsonify,  render_template_string
import os

app = Flask(__name__)


UPLOAD_FOLDER = "/tmp"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# -----------------------------
# Web Upload Page
# -----------------------------
HTML_PAGE = """
<!doctype html>
<title>ASR Upload</title>
<h2>Upload Audio for Transcription</h2>
<form action="/transcribe" method="post" enctype="multipart/form-data">
  <input type="file" name="audio_file" accept=".wav,.mp3" required>
  <br><br>
  <button type="submit">Transcribe</button>
</form>
"""
@app.route("/", methods=["GET"])
def home():
    return render_template_string(HTML_PAGE)

# -----------------------------
# Transcription API
# -----------------------------
@app.route("/transcribe", methods=["POST"])
def transcribe():
    if "audio_file" not in request.files:
        return jsonify({"error": "No audio file uploaded"}), 400

    audio = request.files["audio_file"]
    path = "/content/audio.wav"
    audio.save(path)

    # Validate file size
    if os.path.getsize(path) < 1000:
        return jsonify({"error": "Uploaded file is too small or empty"}), 400

    try:
        segments, info = model.transcribe(path)
        text = " ".join([seg.text for seg in segments])

        return jsonify({
            "language": info.language,
            "transcription": text
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

In [ ]:
import threading

def run_flask():
    app.run(host="0.0.0.0", port=5000)

threading.Thread(target=run_flask).start()

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("38CDZt9pMV9aKWKonNmojynOmCh_7B1hEy6UoDfJ43mjUEDfG")

# Start a tunnel to your Flask app on port 5000
public_url = ngrok.connect(5000)

print("Public API URL:", public_url)